In [112]:
import pandas as pd
import numpy as np
path = "ECAI"

def get_intersection(v01, ve1, v02, ve2):
    # handle division by zero masking out the positions where the denominator is zero
    denominator = ve2 - ve1
    if isinstance(denominator, np.ndarray):
        denominator[denominator == 0] = np.nan
    else:
        if denominator == 0:
            return np.nan
    wes = (v01 - v02) / denominator
    return np.round(wes[(wes >= 0) & (wes <= 10)], 2)

In [113]:
# New method
folder = lambda db, eff, we : f"/db{db}_effrate{eff}_we{we}_ECAI"
unethical = pd.read_csv(path + folder(0, 0.2, 0)*2  + "/mo_rewards.csv")
unethical_np = unethical.to_numpy().reshape(-1, 5, 2)
ethical = pd.read_csv(path + folder(1, 0.2, 10)*2  + "/mo_rewards.csv")
ethical_np = ethical.to_numpy().reshape(-1, 5, 2)

# wes  = get_intersection(ethical_np[:, 0, 0], ethical_np[:, 0, 1], unethical_np[:, 0, 1], unethical_np[:, 0, 1])
wes = get_intersection(unethical_np[:, 0, 0], unethical_np[:, 0, 1], ethical_np[:, 0, 0], ethical_np[:, 0, 1])
wes.mean(), wes.std(), np.median(wes)

(2.4719976019184653, 1.6991766139698001, 2.18)

In [114]:
# Old method
folder = lambda db, eff, we : f"/db{db}_effrate{eff}_we{we}_ECAI"
unethical = pd.read_csv(path + folder(0, 0.2, 0)*2  + "/mo_rewards.csv")
unethical_np = unethical.to_numpy().reshape(-1, 5, 2).mean(axis=0)
ethical = pd.read_csv(path + folder(1, 0.2, 10)*2  + "/mo_rewards.csv")
ethical_np = ethical.to_numpy().reshape(-1, 5, 2).mean(axis=0)

wes  = get_intersection(unethical_np[0, 0], unethical_np[0, 1], ethical_np[0, 0], ethical_np[0, 1])
wes.mean(), wes.std()

(1.75, 0.0)

In [115]:
# New By groups of 500

unethical = pd.read_csv(path + folder(0, 0.2, 0)*2  + "/mo_rewards.csv")
unethical_np = unethical.to_numpy().reshape(-1, 5, 2)
ethical = pd.read_csv(path + folder(1, 0.2, 10)*2  + "/mo_rewards.csv")
ethical_np = ethical.to_numpy().reshape(-1, 5, 2)
unethical_np = unethical_np.reshape(-1, 500, 5, 2)
ethical_np = ethical_np.reshape(-1, 500, 5, 2)

wes = []
for i in range(unethical_np.shape[0]):
    wes.append(get_intersection(unethical_np[i, :, 0, 0], unethical_np[i, :, 0, 1], ethical_np[i, :, 0, 0], ethical_np[i, :, 0, 1]))
    
for we in wes:
    print(we.mean().round(2), "+-", we.std().round(2), "Median" ,np.median(we))

2.37 +- 1.73 Median 2.08
2.53 +- 1.73 Median 2.23
2.48 +- 1.65 Median 2.2
2.49 +- 1.74 Median 2.09
2.42 +- 1.72 Median 2.09
2.34 +- 1.57 Median 2.12
2.52 +- 1.74 Median 2.15
2.59 +- 1.81 Median 2.34
2.47 +- 1.66 Median 2.2
2.5 +- 1.6 Median 2.31


In [116]:
# Old By groups of 500

unethical = pd.read_csv(path + folder(0, 0.2, 0)*2  + "/mo_rewards.csv")
unethical_np = unethical.to_numpy().reshape(-1, 500, 5, 2).mean(axis=1)
ethical = pd.read_csv(path + folder(1, 0.2, 10)*2  + "/mo_rewards.csv")
ethical_np = ethical.to_numpy().reshape(-1, 500, 5, 2).mean(axis=1)

wes = []
for i in range(unethical_np.shape[0]):
    wes.append(get_intersection(unethical_np[i, 0, 0], unethical_np[i, 0, 1], ethical_np[i,0, 0], ethical_np[i, 0, 1]))
    
we = np.array(wes)
we.mean(axis=0), we.std(axis=0), np.median(we, axis=0)

(array([1.756]), array([0.13821722]), array([1.74]))

In [133]:
eff_rates = [0, 0.2, 0.6, 1]
dbs = [0, 1, 10, 100]

we = np.zeros((len(eff_rates), len(dbs), 10))
we_std = np.zeros((len(eff_rates), len(dbs)))
we_median = np.zeros((len(eff_rates), len(dbs)))

for eff_rate in eff_rates:
    for db in dbs:
        if eff_rate == 0:
            continue
        if eff_rate == 1:
            unethical = pd.read_csv(path + folder(0, eff_rate, 10)*2  + "/mo_rewards.csv")
        else:
            unethical = pd.read_csv(path + folder(0, eff_rate, 0)*2  + "/mo_rewards.csv")
        unethical_np = unethical.to_numpy().reshape(-1, 500, 5, 2).mean(axis=1)
        ethical = pd.read_csv(path + folder(db, eff_rate, 10)*2  + "/mo_rewards.csv")
        ethical_np = ethical.to_numpy().reshape(-1, 500, 5, 2).mean(axis=1)

        wes = []
        for i in range(unethical_np.shape[0]):
            wag = []
            for ag in range(5):
                w = get_intersection(unethical_np[i, ag, 0], unethical_np[i, ag, 1], ethical_np[i, ag, 0], ethical_np[i, ag, 1])
                if not np.isnan(w).all():
                    wag.append(w)
            if np.isnan(wag).all():
                continue
            wes.append(np.mean(wag))
        if len(wes) == 0:
            continue
        we[eff_rates.index(eff_rate), dbs.index(db)] = np.array(wes)
        """we[eff_rates.index(eff_rate), dbs.index(db)] = np.array(wes).mean(axis=0).squeeze()
        we_std[eff_rates.index(eff_rate), dbs.index(db)] = np.array(wes).std(axis=0).squeeze()
        we_median[eff_rates.index(eff_rate), dbs.index(db)] = np.median(np.array(wes), axis=0).squeeze()"""
we[eff_rates.index(1), 1:].mean(axis=1)

KeyboardInterrupt: 

In [118]:
we[eff_rates.index(1), 1:].mean(axis=1)

array([0.   , 4.617, 6.597])

In [119]:
for db in dbs:
    print(we[eff_rates.index(0.2), dbs.index(db)].mean())

0.0
1.7559999999999998
1.585
1.9810000000000003


In [120]:
from scipy.stats import f_oneway
 
# Conduct the one-way ANOVA
f_oneway(we[eff_rates.index(0.2), dbs.index(1)], we[eff_rates.index(0.2), dbs.index(10)], we[eff_rates.index(0.2), dbs.index(100)])

F_onewayResult(statistic=23.327105874107495, pvalue=1.306903150069376e-06)